In [1]:
import geopandas as gpd
import ipyleaflet
from ipywidgets import HTML, Layout, FloatSlider, VBox, interact, widgets, Dropdown, Output, HBox, Button, SelectMultiple
from IPython.display import display
from ipyleaflet import Marker, MarkerCluster, Popup, Map, Polygon, Polyline, WidgetControl
import pandas as pd
import numpy as np
from openrouteservice import Client
import openrouteservice.convert as convert
import re

In [2]:
torrevieja_property = pd.read_csv('Torrevieja\Spain-torrevieja_Property_Extended_Match_2023-11-23\Spain-torrevieja_Property_Extended_Match_2023-11-23.csv')
torrevieja_daily = pd.read_csv('Torrevieja\Spain-torrevieja_Daily_Match_2023-11-23\Spain-torrevieja_Daily_Match_2023-11-23.csv')

<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:2: SyntaxWarning: invalid escape sequence '\S'
C:\Users\danim\AppData\Local\Temp\ipykernel_17076\448772125.py:1: SyntaxWarning: invalid escape sequence '\S'
  torrevieja_property = pd.read_csv('Torrevieja\Spain-torrevieja_Property_Extended_Match_2023-11-23\Spain-torrevieja_Property_Extended_Match_2023-11-23.csv')
C:\Users\danim\AppData\Local\Temp\ipykernel_17076\448772125.py:2: SyntaxWarning: invalid escape sequence '\S'
  torrevieja_daily = pd.read_csv('Torrevieja\Spain-torrevieja_Daily_Match_2023-11-23\Spain-torrevieja_Daily_Match_2023-11-23.csv')
C:\Users\danim\AppData\Local\Temp\ipykernel_17076\448772125.py:1: DtypeWarning: Columns (48,56,57,59) have mixed types. Specify dtype option on import or set low_memory=False.
  torrevieja_property = pd.read_csv('Torrevieja\Spain-torrevieja_Property_Extended_Match_2023-11-23\Spain-torreviej

In [ ]:
torrevieja_property.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16196 entries, 0 to 16195
Data columns (total 66 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Property ID                   16196 non-null  object 
 1   Listing Title                 16194 non-null  object 
 2   Property Type                 16192 non-null  object 
 3   Listing Type                  16195 non-null  object 
 4   Created Date                  13670 non-null  object 
 5   Last Scraped Date             16196 non-null  object 
 6   Country                       16195 non-null  object 
 7   State                         14691 non-null  object 
 8   City                          16196 non-null  object 
 9   Currency Native               13666 non-null  object 
 10  Average Daily Rate (USD)      5869 non-null   float64
 11  Average Daily Rate (Native)   5394 non-null   float64
 12  Annual Revenue LTM (USD)      8536 non-null   float64
 13  A

In [3]:
# Función para cerrar el panel
def close_panel_callback(change):
    output_panel.layout.display = "none"  # Ocultar el panel
    m.layout.width = "100%"               # Restaurar el mapa a pantalla completa

In [4]:
# Función para mostrar información en el panel al hacer clic en un marcador
def on_marker_click_output(event=None, type=None, coordinates=None, row=None):
    with details_output:
        details_output.clear_output(wait=True)  # Limpiar el contenido anterior
        display(HTML(f"""
        <h4>Detalles del piso seleccionado</h4>
        <b>Average Daily Rate (Native):</b> {row['Average Daily Rate (Native)']} €<br>
        <b>Annual Revenue LTM (Native):</b> {row['Annual Revenue LTM (Native)']} €<br>
        <b>Property Type:</b> {row['PropertyType']}<br>
        <b>Habitaciones:</b> {row['Bedrooms']}<br>
        <b>Baños:</b> {row['Bathrooms']}<br>
        <b>Max Guests:</b> {row['Max Guests']}<br>
        <b>Overall Rating:</b> {row['Overall Rating']}<br>
        <b>URL:</b> {row['Listing URL']}
        """))
    # Mostrar el panel flotante
    output_panel.layout.display = "flex"  # Mostrar el panel
    m.layout.width = "70%"               # Reducir el ancho del mapa para que quepa el panel
    # Actualizar los botones con las rutas del piso seleccionado
    # update_buttons(row)

In [5]:
# Función para extraer latitud y longitud
def extract_coordinates(point):
    # Usar una expresión regular para extraer los números
    match = re.match(r'POINT \(([\d\.\-]+) ([\d\.\-]+)\)', point)
    if match:
        # Retorna los números como flotantes
        return float(match.group(1)), float(match.group(2))
    return None, None  # En caso de error

In [6]:
# Función para actualizar el mapa según el barrio seleccionado
def update_map(change):
    selected_pisos = pisos  # DataFrame de pisos

    # Limpiar los marcadores actuales del mapa
    for marker in markers:
        m.remove_layer(marker)
    markers.clear()

    filtered_pisos = selected_pisos

    # Añadir los marcadores de los pisos filtrados al mapa
    for _, row in filtered_pisos.iterrows():
        lat, lon = row["LATITUDE"], row["LONGITUDE"]
        marker = ipyleaflet.Marker(location=(lat, lon), draggable=False)
        marker.on_click(lambda event, type, coordinates, row=row: on_marker_click_output(event, type, coordinates, row))
        markers.append(marker)
        m.add_layer(marker)

In [ ]:
# Crear el mapa interactivo centrado en Valencia
torrevieja_centro = (np.mean(torrevieja_property["Latitude"]), np.mean(torrevieja_property["Longitude"]))
m = ipyleaflet.Map(center=torrevieja_centro, zoom=15, layout=Layout(width="100%", height="100vh"), scroll_wheel_zoom=True)

# Dibujar los límites de los barrios en el mapa
# barrios_polygons = []
# for _, barrio in barrios_gdf.iterrows():
#     geometry = barrio["geometry"]
    
#     # Si es un polígono o multipolígono, lo añadimos correctamente
#     if geometry.geom_type == "Polygon":
#         locations = [[(coord[1], coord[0]) for coord in geometry.exterior.coords]]
#         polygon = Polygon(
#             locations=locations,
#             color="black",
#             fill_color="gray",
#             fill_opacity=0.2,
#             name=barrio["name"]
#         )
#         m.add_layer(polygon)
#         barrios_polygons.append(polygon)
    
#     elif geometry.geom_type == "MultiPolygon":
#         for poly in geometry.geoms:
#             locations = [[(coord[1], coord[0]) for coord in poly.exterior.coords]]
#             polygon = Polygon(
#                 locations=locations,
#                 color="black",
#                 fill_color="gray",
#                 fill_opacity=0.2,
#                 name=barrio["name"]
#             )
#             m.add_layer(polygon)
#             barrios_polygons.append(polygon)
    
#     # Si es un LineString, lo mostramos como una línea
#     elif geometry.geom_type == "LineString":
#         locations = [(coord[1], coord[0]) for coord in geometry.coords]
#         polyline = Polyline(
#             locations=locations,
#             color="black",
#             weight=2,
#             fill=False
#         )
#         m.add_layer(polyline)

# Crear un selector múltiple para los barrios
# barrio_selector = SelectMultiple(
#     options=["Todos"] + list(barrios_gdf["name"].unique()),  # Lista de barrios
#     value=["Todos"],
#     description="Barrios:",
#     layout=Layout(width="200px", height="150px"),  # Tamaño ajustado
#     style={"description_width": "initial"}
# )


# Crear el panel lateral (Output widget)
output_panel = VBox(
    layout=Layout(
        overflow='scroll hidden',
        display="none",       # Oculto inicialmente
        width="30%",          # Ancho del panel (30% del contenedor)
        height="100%",        # Altura completa
        background_color="white",
        border="1px solid #ccc",
        padding="10px",
    )
)

# Crear un botón para cerrar el panel
close_button = Button(
    description="Cerrar",
    layout=Layout(width="100%"),
    button_style="danger",  # Color rojo para indicar cierre
)

# Crear los botones iniciales
# school_button = create_button_with_style("Colegio", "graduation-cap", "blue", "Mostrar la ruta al colegio más cercano")
# hospital_button = create_button_with_style("Hospital", "hospital", "red", "Mostrar la ruta al hospital más cercano")
# park_button = create_button_with_style("Parque", "tree", "green", "Mostrar la ruta al parque más cercano")

# Contenedor para los detalles del piso
details_output = Output()

# Añadir el botón cerrar y los botones de rutas al panel
output_panel.children = [close_button, details_output, school_button, hospital_button, park_button]

close_button.on_click(close_panel_callback)

# Crear una lista de marcadores para los pisos
markers = []

# Lista para almacenar las rutas actuales
current_routes = []

# Conectar el selector de barrios con la función de actualización
barrio_selector.observe(update_map, names="value")

# Añadir el selector al mapa como un WidgetControl en la parte superior izquierda
barrio_selector_control = WidgetControl(widget=barrio_selector, position="topleft")  # Cambia "topright" por "topleft" si es necesario
m.add_control(barrio_selector_control)

# Contenedor principal (mapa y panel superpuestos)
ui = HBox([m, output_panel])
ui.layout = Layout(width="100%", height="100vh")

# Mostrar la interfaz
display(ui)